In [ ]:
import os
from pyspark.sql import SparkSession

MINIO_ACCESS_KEY = "SnAAJOVJ7cszy3eDFDei"
MINIO_SECRET_KEY = "4Q3PJigjEpVZVRPAzv1n89VFQATq9Zu9oxzSjKFg"

In [ ]:
os.makedirs("/tmp/spark-logs", exist_ok=True)

In [ ]:
%%time
spark = SparkSession.builder.master("spark://spark-master:7077") \
    .appName("MyAppTuning") \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "file:/tmp/spark-logs") \
    .config("spark.history.fs.logDirectory", "file:/tmp/spark-logs") \
    .config("log4j.rootCategory", "INFO, console") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,io.delta:delta-core_2.12:2.4.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "1536m") \
    .config("spark.driver.memory", "1536m") \
    .config("spark.sql.shuffle.partitions", "16") \
    .config("spark.storage.memoryFraction", "0.4") \
    .config("spark.shuffle.memoryFraction", "0.5") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "512m") \
    .config("spark.sql.parquet.compression.codec", "gzip") \
    .config("spark.sql.orc.compression.codec", "zlib") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC") \
    .getOrCreate()

In [ ]:
sc = spark.sparkContext
hadoop_conf = sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.access.key", MINIO_ACCESS_KEY)
hadoop_conf.set("fs.s3a.secret.key", MINIO_SECRET_KEY)
hadoop_conf.set("fs.s3a.endpoint", "http://minio:9000")
hadoop_conf.set("fs.s3a.path.style.access", "true")
hadoop_conf.set("fs.s3a.connection.ssl.enabled", "false")

In [ ]:
spark

In [ ]:
minio_bucket_bronze = "bronze"

In [ ]:
df_delta = spark.read.format('delta').load(f"s3a://{minio_bucket_bronze}/test/delta")

In [ ]:
%%time
df_delta.count()

In [ ]:
%%time
df_delta.select(['hotel', 'children', 'country']).limit(10).show(truncate=False)

In [ ]:
spark.stop()